## Eulerian Fluid Simulation
*still sensors that never moves*
- Eulerian representation uses still sensors in space, usually arranged in a
 regular grid/triangular mesh.
- A bit of math - but not too much.

### Overview
#### Material Derivatives: Lagrangian v.s. Eulerian (材料导数，物质导数)

$
\displaystyle\frac{D}{Dt} := \frac{\partial}{\partial t}+{\rm u}\cdot\nabla
$

E.g.,<br>
$\begin{array}{rcl}
\displaystyle\frac{DT}{Dt} & = & \displaystyle\frac{\partial T}{\partial t}+{\rm u}\cdot\nabla T \\
\displaystyle\frac{D{\rm u}_{x}}{Dt} & = & \displaystyle\frac{\partial {\rm u}_{x}}{\partial t}+{\rm u}\cdot\nabla {\rm u}_{x}
\end{array}$
<br>

${\rm u}$: material (fluid) velocity. Many other names: Advective/Lagrangian/particle derivative.
Intuitively, change of physical quantity on a piece of material =

1. change due to time $\frac{\partial}{\partial t}$ (Eulerian).
2. change due to material movement ${\rm u}\cdot\nabla$.

#### (Incompressible) Navier-Stokes equations
$
\begin{array}{rcl}
\rho\displaystyle\frac{D{\rm u}}{Dt}&=&-\nabla p+\mu\nabla^{2}{\rm u}+\rho g \\
\displaystyle\frac{D{\rm u}}{Dt}&=&-\displaystyle\frac{1}{\rho}\nabla p+\nu\nabla^{2}{\rm u}+g \\
\nabla \cdot {\rm u}&=&0
\end{array}$
<br>

$\mu$: dynamic viscosity; $\nu=\frac{\mu}{\rho}$: kinematic viscosity.<br>
In graphics we usually drop **viscosity** except for highly viscous materials (e.g., honey)

#### Operator splitting [[More Details](./operator-splitting-wotaoyinsplittingbookch3-macnamara-strangpdf-operator.pdf)]
$
\begin{array}{rclc}
\displaystyle\frac{D{\rm u}}{Dt}&=&-\displaystyle\frac{1}{\rho}\nabla p+g,&\nabla \cdot {\rm u}=0
\end{array}$
<br>
Split the equations above into three parts:

$
\begin{array}{rcllr}
\displaystyle\frac{D{\rm u}}{Dt}&=&0,\displaystyle\frac{D\alpha}{Dt}=0&(\mathbf{advection})&(1) \\
\displaystyle\frac{\partial{\rm u}}{\partial t}&=&g&({\rm external forces, optional})&(2) \\
\displaystyle\frac{\partial{\rm u}}{\partial t}&=&-\displaystyle\frac{1}{\rho}\nabla p\qquad s.t.\quad\nabla \cdot {\rm u}=0&(\mathbf{projection})&(3)
\end{array}$
<br>

$\alpha$: any physical property (temperature, color, smoke density etc.)

#### Eulerian ﬂuid simulation cycle
Time discretization with splitting: for each time step,<br>
1. Advection: “move” the ﬂuid feld. Solve ${\rm u}^{*}$ using  ${\rm u}^{t}$
$
\begin{equation}
\displaystyle\frac{D{\rm u}}{Dt}=0,\displaystyle\frac{D\alpha}{Dt}=0
\end{equation}
$
2. External forces (optional): evaluate ${\rm u}^{**}$ using ${\rm u}^{*}$
$
\begin{equation}
\displaystyle\frac{\partial{\rm u}}{\partial t}=g
\end{equation}
$
3. Projection: make velocity field ${\rm u}^{t+1}$ dicergence-free based on ${\rm u}^{**}$
$
\begin{equation}
\displaystyle\frac{\partial{\rm u}}{\partial t}=-\displaystyle\frac{1}{\rho}\nabla p\qquad s.t.\quad\nabla \cdot{\rm u}^{t+1}=0
\end{equation}
$


### Grid
#### Spatial discretization using cell-centered grids
![fig](./cell-centered-grigs.png)<br>
Figure: ${\rm u}_{x},{\rm u}_{y},p$ are all stored at the center (orange) of cells

```python
import taichi as ti
n, m = 3, 3
u = ti.var(ti.f32, shape=(n, m)) # x-component of velocity
v = ti.var(ti.f32, shape=(n, m)) # y-component of velocity
p = ti.var(ti.f32, shape=(n, m)) # pressure
```

#### Spatial discretization using staggered grids
![fig](./staggered-grids.png)<br>
Figure: Red: ${\rm u}_{x}$; Green: ${\rm u}_{y}$; Orange: $p$.
```python
import taichi as ti
n, m = 3, 3
u = ti.var(ti.f32, shape=(n+1, m)) # x-component of velocity
v = ti.var(ti.f32, shape=(n, m+1)) # y-component of velocity
p = ti.var(ti.f32, shape=(n, m)) # pressure
```

#### Bilinear interpolation (双线性插值)
![Bilinear interpolation](./bilinearInt.png)<br>
Figure: Bilibnear interpolation: value at $(x,y)$ is a weighted average of the four corners.**Source: [Wikipedia](https://en.wikipedia.org/wiki/Bilinear_interpolation)**

### Advection (流动过程)
#### Advection schemes
A trade-ff between numerical viscosity, stability, performance and complexity:
- Semi-Lagrangian advection
- MacCormack/BFECC
- BiMocq
- Particle advection (PIC/FLIP/APIC/PolyPIC, later in this course)
- ...

#### Semi-Lagrangian advection
![SLA](./semi-lagrangian.png)<br>
Figure: What should be the field value atpnow based on the field and velocity at theprevious time step? Well, just let reverse the simulation...

```python
import taichi as ti
@ti.func
def semi_lagrangian(x, new_x, dt):
    for I in ti.grouped(x):
        new_x[I] = sample_bilinear(x, backtrace(I, dt))
```

#### What if...
![Whatif](what-if.png)<br>
Figure: The real trajectory of material parcels can be complex... Red: a naive estimationof last position; Light gree: the true previous position.


In [2]:
import taichi as ti

ti.init(arch=ti.gpu)

use_mc = False
mc_clipping = False
pause = False

# Runge-Kutta order
rk = 3

n = 512
x = ti.field(ti.f32, shape=(n, n))
new_x = ti.field(ti.f32, shape=(n, n))
new_x_aux = ti.field(ti.f32, shape=(n, n))
dx = 1 / n
inv_dx = 1 / dx
dt = 0.05

stagger = ti.Vector([0.5, 0.5])


@ti.func
def Vector2(x, y):
    return ti.Vector([x, y])


@ti.func
def inside(p, c, r):
    return (p - c).norm_sqr() <= r * r


@ti.func
def inside_taichi(p):
    p = Vector2(0.5, 0.5) + (p - Vector2(0.5, 0.5)) * 1.2
    ret = -1
    if not inside(p, Vector2(0.50, 0.50), 0.55):
        if ret == -1:
            ret = 0
    if not inside(p, Vector2(0.50, 0.50), 0.50):
        if ret == -1:
            ret = 1
    if inside(p, Vector2(0.50, 0.25), 0.09):
        if ret == -1:
            ret = 1
    if inside(p, Vector2(0.50, 0.75), 0.09):
        if ret == -1:
            ret = 0
    if inside(p, Vector2(0.50, 0.25), 0.25):
        if ret == -1:
            ret = 0
    if inside(p, Vector2(0.50, 0.75), 0.25):
        if ret == -1:
            ret = 1
    if p[0] < 0.5:
        if ret == -1:
            ret = 1
    else:
        if ret == -1:
            ret = 0
    return ret


@ti.kernel
def paint():
    for i, j in ti.ndrange(n * 4, n * 4):
        ret = 1 - inside_taichi(Vector2(i / n / 4, j / n / 4))
        x[i // 4, j // 4] += ret / 16


@ti.func
def velocity(p):
    return ti.Vector([p[1] - 0.5, 0.5 - p[0]])


@ti.func
def vec(x, y):
    return ti.Vector([x, y])


@ti.func
def clamp(p):
    for d in ti.static(range(p.n)):
        p[d] = min(1 - 1e-4 - dx + stagger[d] * dx, max(p[d], stagger[d] * dx))
    return p


@ti.func
def sample_bilinear(x, p):
    p = clamp(p)

    p_grid = p * inv_dx - stagger

    I = ti.cast(ti.floor(p_grid), ti.i32)
    f = p_grid - I
    g = 1 - f

    return x[I] * (g[0] * g[1]) + x[I + vec(1, 0)] * (
            f[0] * g[1]) + x[I + vec(0, 1)] * (
                   g[0] * f[1]) + x[I + vec(1, 1)] * (f[0] * f[1])


@ti.func
def sample_min(x, p):
    p = clamp(p)
    p_grid = p * inv_dx - stagger
    I = ti.cast(ti.floor(p_grid), ti.i32)

    return min(x[I], x[I + vec(1, 0)], x[I + vec(0, 1)], x[I + vec(1, 1)])


@ti.func
def sample_max(x, p):
    p = clamp(p)
    p_grid = p * inv_dx - stagger
    I = ti.cast(ti.floor(p_grid), ti.i32)

    return max(x[I], x[I + vec(1, 0)], x[I + vec(0, 1)], x[I + vec(1, 1)])


@ti.func
def backtrace(I, dt):
    p = (I + stagger) * dx
    if ti.static(rk == 1):
        p -= dt * velocity(p)
    elif ti.static(rk == 2):
        p_mid = p - 0.5 * dt * velocity(p)
        p -= dt * velocity(p_mid)
    elif ti.static(rk == 3):
        v1 = velocity(p)
        p1 = p - 0.5 * dt * v1
        v2 = velocity(p1)
        p2 = p - 0.75 * dt * v2
        v3 = velocity(p2)
        p -= dt * (2 / 9 * v1 + 1 / 3 * v2 + 4 / 9 * v3)
    else:
        ti.static_print(f"RK{rk} is not supported.")

    return p


@ti.func
def semi_lagrangian(x, new_x, dt):
    # Note: this loop is parallelized
    for I in ti.grouped(x):
        new_x[I] = sample_bilinear(x, backtrace(I, dt))


# Reference: https://github.com/ziyinq/Bimocq/blob/master/src/bimocq2D/BimocqSolver2D.cpp

@ti.func
def maccormack(x, dt):
    semi_lagrangian(x, new_x, dt)
    semi_lagrangian(new_x, new_x_aux, -dt)

    for I in ti.grouped(x):
        new_x[I] = new_x[I] + 0.5 * (x[I] - new_x_aux[I])

        if ti.static(mc_clipping):
            source_pos = backtrace(I, dt)
            min_val = sample_min(x, source_pos)
            max_val = sample_max(x, source_pos)

            if new_x[I] < min_val or new_x[I] > max_val:
                new_x[I] = sample_bilinear(x, source_pos)


@ti.kernel
def advect():
    if ti.static(use_mc):
        maccormack(x, dt)
    else:
        semi_lagrangian(x, new_x, dt)

    for I in ti.grouped(x):
        x[I] = new_x[I]


paint()

gui = ti.GUI('Advection schemes', (512, 512))

while not gui.get_event(gui.ESCAPE):
    while gui.get_event(ti.GUI.PRESS):
        if gui.event.key in [ti.GUI.ESCAPE, ti.GUI.EXIT]: exit(0)
        if gui.event.key == ti.GUI.SPACE:
            pause = not pause
    if not pause:
        for i in range(1):
            advect()
    if not gui.running:
        break
    gui.set_image(x.to_numpy())
    gui.show()
    
from cv2 import imshow
imshow(x)
print("exit while loop")

[Taichi] Starting on arch=cuda
[Taichi] materializing...


error: OpenCV(4.5.3) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)
